In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pickle

In [4]:
lc=pd.read_csv("LendingClub.csv")
# tit.describe() lc.dtypes() 
lc.head()

,ID,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,2,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,3,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,4,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,5,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [12]:
lc["int_rate"].describe()

count    9578.000000
mean        0.122640
std         0.026847
min         0.060000
25%         0.103900
50%         0.122100
75%         0.140700
max         0.216400
Name: int_rate, dtype: float64

In [5]:
lc.corr()['not_fully_paid'].sort_values()

credit_policy       -0.158119
fico                -0.149666
log_annual_inc      -0.033439
days_with_cr_line   -0.029237
delinq_2yrs          0.008881
dti                  0.037362
pub_rec              0.048634
installment          0.049955
revol_bal            0.053699
ID                   0.081646
revol_util           0.082088
inq_last_6mths       0.149452
int_rate             0.159552
not_fully_paid       1.000000
Name: not_fully_paid, dtype: float64

In [7]:
# #df = pd.get_dummies(lc, columns=['purpose']).drop(columns=['purpose_debt_consolidation'])

# X = df.drop(columns=['not_fully_paid'])
# y = df['not_fully_paid']

In [8]:

lc.head()

,ID,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,2,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,3,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,4,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,5,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [9]:
lc.isna().sum()

ID                   0
credit_policy        0
purpose              0
int_rate             0
installment          0
log_annual_inc       0
dti                  0
fico                 0
days_with_cr_line    0
revol_bal            0
revol_util           0
inq_last_6mths       0
delinq_2yrs          0
pub_rec              0
not_fully_paid       0
dtype: int64

In [10]:
lc.not_fully_paid.value_counts()

0    8045
1    1533
Name: not_fully_paid, dtype: int64

In [11]:
# lc_0 = lc[df.not_fully_paid==0]
# lc_1 = lc[df.not_fully_paid==1]

# n_minority_class = lc_1.shape[0]
# n_minority_class

NameError: name 'df' is not defined

In [ ]:
from sklearn.utils import resample
                                                                  
lc_0= resample(lc_0, replace=False,                  
                             n_samples=n_minority_class, 
                             random_state=1)
lc_0.shape

(1533, 14)

In [ ]:
lc_new = pd.concat([lc_0, lc_1])
lc_new.not_fully_paid.value_counts()

0    1533
1    1533
Name: not_fully_paid, dtype: int64

In [ ]:
lc_new.nunique() # nunique()shows how many categorical values for each attribute

credit_policy           2
purpose                 7
int_rate              238
installment          2188
log_annual_inc        875
dti                  1761
fico                   42
days_with_cr_line    1351
revol_bal            2799
revol_util            967
inq_last_6mths         24
delinq_2yrs             7
pub_rec                 4
not_fully_paid          2
dtype: int64

In [ ]:
#tit.pclass.unique() #unique()shows categorical values for each attribute

array([1, 2, 3])

In [ ]:
# building the pipline
X = lc_new.drop(columns=['not_fully_paid'])
y = lc_new['not_fully_paid']

In [ ]:
num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler()),
])

full_pipeline = ColumnTransformer([
    ('num',num_pipeline,['int_rate','installment','log_annual_inc','dti',
                         'fico','days_with_cr_line','revol_bal','revol_util','inq_last_6mths','delinq_2yrs','pub_rec']),
    ('cat',preprocessing.OneHotEncoder(),['credit_policy','purpose'])     #OneHotEncoder()change the text to numeric variable like 00001.
])

In [ ]:
lc_X_tr=full_pipeline.fit_transform(X)

pd.DataFrame(lc_X_tr) # 多重共线性package会包会自己解决，不需要删除某列

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.762610,-0.663220,-0.279264,-0.359276,1.895253,0.019344,-0.397108,-1.349571,-0.739504,-0.307965,-0.279399,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.249167,-0.144102,0.574586,0.939358,-0.640894,1.798468,0.193161,0.720149,-0.339059,-0.307965,3.335100,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.229759,-0.346031,-0.682026,-0.297709,1.227846,3.674756,-0.243309,-0.731388,0.461831,-0.307965,-0.279399,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-1.762610,0.798215,0.544799,0.309370,2.028735,1.715211,-0.108994,-1.141233,-0.739504,-0.307965,-0.279399,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.319473,-0.766660,-1.457323,-1.537641,-0.106968,-0.327957,-0.401233,-1.096833,0.862276,-0.307965,-0.279399,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061,0.727726,0.070628,1.976712,-0.346390,-0.907857,2.362020,4.558455,1.126579,0.061386,-0.307965,-0.279399,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3062,-0.041947,-0.336126,0.341804,-1.803954,0.426958,-0.065108,-0.427368,-1.639879,1.262720,-0.307965,-0.279399,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3063,-0.715411,-1.083149,-0.516220,0.040193,-0.507412,-0.435493,-0.199101,1.153902,2.464055,-0.307965,-0.279399,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3064,1.242074,0.102491,-0.165059,0.912154,-0.373931,-1.092675,-0.431631,-1.568156,1.262720,-0.307965,-0.279399,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
pickle.dump(full_pipeline,open('feature.pipeline','wb'))

In [ ]:
# select logistic regression when outcome is categorical variable
log_reg = LogisticRegression()
scores= cross_val_score(log_reg,lc_X_tr,y,scoring='accuracy',cv=5) # cv交叉验证迭代次数
scores.mean()

0.5404721798598232

In [ ]:
log_reg.fit(lc_X_tr,y)
pickle.dump(log_reg,open('lc.model','wb'))

In [ ]:
# saving and loading model to use it on another dataset
# import pickle                                          from joblib import dump load
# s= pickle.dumps(clf) clf2=pickle.loads(s)               fump(clf,'filename.joblib') clf=load('filename.joblib)

